In [1]:
print('hi')

hi


In [2]:
import pandas as pd
import torch

papers=pd.read_json('submission_metadata.json').T
papers.index.name = 'row_id'

malicious=pd.read_json('101ranking.json')
authors=pd.read_json('authors92K.json')

papers_tensor=torch.load('papers.pt')
malicious_tensor=torch.load('malicious.pt')
authors_tensor=torch.load('authors.pt')
rewritten_tensor=torch.load('rewritten.pt')

print(authors_tensor.shape)
print(papers_tensor.shape)
print(malicious_tensor.shape)
print(rewritten_tensor.shape)


torch.Size([92819, 768])
torch.Size([3218, 768])
torch.Size([300, 768])
torch.Size([300, 768])


In [3]:
from torch.nn.functional import cosine_similarity

def my_cosine_similarity(paper,list_of_papers): 
    """
    Calcul la similarité cosinus entre un papier et une les papiers d'un auteur
    """
    rez=[]
    for paper2 in list_of_papers:
        rez.append(cosine_similarity(paper,paper2,dim=0))
    return rez

def get_similarity(author, paper, mode='max'):
        similarities = my_cosine_similarity(paper,authors_dico[author])
        if mode== "max":
            return max(similarities)
        elif mode == "avg":
            return torch.stack(similarities).mean(dim=0)


In [ ]:
i=0
authors_dico={}
for index,row in authors.iterrows():
    if(row['AuthorID'] not in authors_dico):
        authors_dico[row['AuthorID']]=[] #si l'auteur est pas dans le dico, on lui crée une liste vide
    authors_dico[row['AuthorID']].append(authors_tensor[i]) 
    i+=1

targets=[]
for elt in malicious['colluding_reviewer_profile_link']:
    targets.append(elt[34:])
malicious['target'] = targets

liste_ids=[]
for id, row in malicious.iterrows():
  for index,d in papers.iterrows():
    if(d['title']==row['title']):
      liste_ids.append(index)

malicious['id']=liste_ids

In [ ]:
#il ne faut garder que les bons auteurs ! ceux qui ont été des targets.
good_authors={}
i=0
for id,elt in malicious.iterrows():
    if(elt['target'] in good_authors.keys()):
        print(i) #des auteurs sont targets plusieurs fois
        i+=1
    good_authors[elt['target']] = authors_dico[elt['target']]

In [ ]:
dico_rankings={}
i=0
for author_id,author_tens in good_authors.items():
    dico_rankings[author_id]={}
    for paper_tens,paper_metadata in zip(papers_tensor,papers.iterrows()):
        dico_rankings[author_id][paper_metadata[0]]=get_similarity(author_id,paper_tens,mode='avg')
    i+=1
    print(i)


In [27]:
for key,value in dico_rankings.items():
    for author_id,author_tens in value.items():
        value[author_id]=float(author_tens) #transforme les tensors en floats psk c'est pas serialisable pour en faire un JSON
import json
with open("dico_rankings.json", "w") as f:
    json.dump(dico_rankings, f)

In [28]:
import json
with open("dico_rankings.json", "r") as f:
    my_dict = json.load(f)


In [42]:
for id,row in malicious.iterrows():
    my_dict[row['target']]['ORIGINAL']=my_dict[row['target']].pop(row['id'])

#for key,value in my_dict.items(): #On a remplacé cible ID par ORIGINAL
    #print(value['ORIGINAL'])

In [43]:
def find_name_rank(name,liste):
    for i,elt in enumerate(liste):
        if(name==elt[0]): 
            return i

In [44]:
id=0
for id,row in malicious.iterrows():
    similarity_score=get_similarity(row['target'],malicious_tensor[id],mode='avg')
    my_dict[row['target']]['MALICIOUS']=similarity_score
    similarity_score_rewritten=get_similarity(row['target'],rewritten_tensor[id],mode='avg')
    my_dict[row['target']]['REWRITTEN']=similarity_score_rewritten

In [46]:
dict_sorted={}
for key,value in my_dict.items():
    dict_sorted[key]=sorted(value.items(), key=lambda x: x[1], reverse=True)

In [47]:
moyenne_ORIGINAL=[]
moyenne_MALICIOUS=[]
moyenne_REWRITTEN=[]

for id,row in malicious.iterrows():
    a=find_name_rank('ORIGINAL',dict_sorted[row['target']])
    b=find_name_rank('MALICIOUS',dict_sorted[row['target']])
    c=find_name_rank('REWRITTEN',dict_sorted[row['target']])
    
    moyenne_ORIGINAL.append(a)
    moyenne_MALICIOUS.append(b)
    moyenne_REWRITTEN.append(c)


In [59]:
compteur=0
for i in range(0,len(moyenne_MALICIOUS)):
    if(moyenne_MALICIOUS[i]<=5):
        compteur+=1
print(compteur)

113


In [49]:
for a,b,c in zip(moyenne_ORIGINAL,moyenne_MALICIOUS,moyenne_REWRITTEN):
    pass
    #print(a,b,c)

ORIGINAL_MEAN=sum(moyenne_ORIGINAL)/len(moyenne_ORIGINAL)
MALICIOUS_MEAN=sum(moyenne_MALICIOUS)/len(moyenne_MALICIOUS)
REWRITTEN_MEAN=sum(moyenne_REWRITTEN)/len(moyenne_REWRITTEN)

In [50]:
ORIGINAL_MEAN-MALICIOUS_MEAN

209.3066666666667

In [54]:
REWRITTEN_MEAN

513.9766666666667

In [52]:
ORIGINAL_MEAN

260.4066666666667

In [53]:
MALICIOUS_MEAN

51.1